# Daily Greeting - Prompt Iteration

This notebook lets you iterate on prompting techniques with cached weather and literature data.

In [1]:
import json
import random
import logging
from daily_greeting import send_ollama_request
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

## Fetch Weather Data

In [2]:
# Fetch fresh data
from daily_greeting import get_weather_data
weather = get_weather_data()

logging.info("Weather:")
logging.info(json.dumps(weather, indent=2))

INFO: Fetching forecast url for coordinates: 42.2688, -71.8088
INFO: Fetching forecast data...
INFO: Weather:
INFO: {
  "overnight": {
    "dayOfWeek": "Tonight",
    "precipitation": 0,
    "description": "Clear, with a low around 51. West wind around 9 mph."
  },
  "sunrise": {
    "temperature": 51,
    "humidity": 93,
    "dewpoint": 9.444444444444445,
    "windSpeed": "8 mph",
    "windDirection": "W",
    "precipitation": 0,
    "conditions": "Sunny"
  },
  "today": {
    "dayOfWeek": "Saturday",
    "precipitation": 2,
    "description": "Sunny, with a high near 79. West wind around 8 mph."
  }
}


# Fetch Literature Data

Run it through the LLM up to some number of times to see whether it thinks it is ineteresting enough to use.

In [3]:
from daily_greeting import get_random_literature, format_literature

attempts = 0
logging.info("Starting literature selection process.")
while attempts < 5:
    literature = get_random_literature()
    if not literature:
        logging.warning("No literature found, trying again...")
        attempts += 1
        continue

    formatted_lit = format_literature(literature)
    literature_prompt = f"""Please evaluate whether the following literary excerpt is suitable material from which to source themes, tone, imagery, or metaphor.

{formatted_lit}

Respond in this exact format:
REASONING: One sentence reasoning about the suitability of the text.
VERDICT: YES or NO"""

    print("\n" + "="*50)
    print("LITERATURE EVALUATION PROMPT")
    print("="*50)
    print(literature_prompt)

    evaluation = send_ollama_request(literature_prompt.format())

    print("\n" + "="*50)
    print("LITERATURE EVALUATION")
    print("="*50)
    print(evaluation)

    attempts += 1
    if "VERDICT: YES" in evaluation.upper():
        break

if attempts == 5:
    logging.error("Failed to find suitable literature after 5 attempts.")
else:
    logging.info(f"Suitable literature found in {attempts} attempts.")

logging.info("\nLiterature:")
logging.info(json.dumps(literature, indent=2))

INFO: Starting literature selection process.
INFO: Fetching books from Gutendex page 21...
INFO: Found 32 books on page 21.
INFO: Fetching book content of book...
INFO: Sending request to Ollama model mistral:7b...



LITERATURE EVALUATION PROMPT
Please evaluate whether the following literary excerpt is suitable material from which to source themes, tone, imagery, or metaphor.

Literature excerpt:
"The Younger Edda; Also called Snorre's Edda, or The Prose Edda" by Snorri Sturluson:
that he first would like to ask whether
there was any wise man. Answered Har: You will not come out from here
hale unless you are wiser.

  And stand now forth
  While you ask;
  He who answers shall sit.




CHAPTER III.

OF THE HIGHEST GOD.


3. Ganglere then made the following question: Who is the highest and
oldest of all the gods? Made answer Har: Alfather he is called in our
tongue, but in Asgard of old he had twelve names. The first is Alfather,
the second is Herran or Herjan, the third Nikar or Hnikar, the fourth
Nikuz or Hnikud, the fifth Fjolner, the sixth Oske, the seventh Ome, the
eighth Biflide or Biflinde, the ninth Svidar, the tenth Svidrer, the
eleventh Vidrer, the twelfth Jalg or Jalk. Ganglere asks agai

INFO: Suitable literature found in 1 attempts.
INFO: 
Literature:
INFO: {
  "title": "The Younger Edda; Also called Snorre's Edda, or The Prose Edda",
  "author": {
    "name": "Snorri Sturluson",
    "birth_year": null,
    "death_year": 1241
  },
  "excerpt": "that he first would like to ask whether\r\nthere was any wise man. Answered Har: You will not come out from here\r\nhale unless you are wiser.\r\n\r\n  And stand now forth\r\n  While you ask;\r\n  He who answers shall sit.\r\n\r\n\r\n\r\n\r\nCHAPTER III.\r\n\r\nOF THE HIGHEST GOD.\r\n\r\n\r\n3. Ganglere then made the following question: Who is the highest and\r\noldest of all the gods? Made answer Har: Alfather he is called in our\r\ntongue, but in Asgard of old he had twelve names. The first is Alfather,\r\nthe second is Herran or Herjan, the third Nikar or Hnikar, the fourth\r\nNikuz or Hnikud, the fifth Fjolner, the sixth Oske, the seventh Ome, the\r\neighth Biflide or Biflinde, the ninth Svidar, the tenth Svidrer, the\r\nel


LITERATURE EVALUATION
 REASONING: The excerpt from "The Younger Edda" by Snorri Sturluson is a suitable source for themes, tone, imagery, and metaphor as it provides insights into Norse mythology, showcases the dialogue between characters that can reveal tone and character development, uses descriptive language to convey imagery, and contains metaphors such as the twelve names of the highest god.

VERDICT: YES


# Fetch Navidrome Album

Request five random albums from the Navidrome server. Select one which will pair best with the selected literature.

In [4]:
from daily_greeting import get_navidrome_albums, format_albums

albums = get_navidrome_albums(count=5)
formatted_albums = format_albums(albums)

album_prompt = f"""Please evaluate which of the following albums would pair most interestingly with the selected literary excerpt, whether by contrast or by complement.

{formatted_albums}

{formatted_lit}

Respond in this exact format:
REASONING: Two or three sentences reasoning about the best album choice.
VERDICT: [N]"""

print("\n" + "="*50)
print("ALBUM SELECTION PROMPT")
print("="*50)
print(album_prompt)

album_evaluation = send_ollama_request(album_prompt)

print("\n" + "="*50)
print("ALBUM SELECTION")
print("="*50)
print(album_evaluation)

INFO: Fetching random albums from Navidrome server...
INFO: Sending request to Ollama model mistral:7b...



ALBUM SELECTION PROMPT
Please evaluate which of the following albums would pair most interestingly with the selected literary excerpt, whether by contrast or by complement.

Albums:
[1] "Piano Recital" by Paul Baumgartner (Unknown) - Genres: Keyboard
[2] "Mellon Collie and the Infinite Sadness - TwilightTo Starlight" by Smashing Pumpkins (1995) - Genres: Rock
[3] "Avalon (DSD Mix)" by Roxy Music (1982) - Genres: Pop
[4] "Selected Ambient Works 85-92" by Aphex Twin (Unknown) - Genres: Ambient
[5] "Parachutes" by Coldplay (Unknown) - Genres: Alternative

Literature excerpt:
"The Younger Edda; Also called Snorre's Edda, or The Prose Edda" by Snorri Sturluson:
that he first would like to ask whether
there was any wise man. Answered Har: You will not come out from here
hale unless you are wiser.

  And stand now forth
  While you ask;
  He who answers shall sit.




CHAPTER III.

OF THE HIGHEST GOD.


3. Ganglere then made the following question: Who is the highest and
oldest of all the go